In [2]:
# Import PySpark
from pyspark.sql import SparkSession

#Create SparkSession
spark = SparkSession.builder.appName('MyApp').getOrCreate()

23/03/15 11:14:15 WARN Utils: Your hostname, MacBook-Air-di-Emanuele.local resolves to a loopback address: 127.0.0.1; using 10.0.0.197 instead (on interface en0)
23/03/15 11:14:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/15 11:14:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
"""
Loading the training csv.
"""

training = spark.read.option("escape", "\"").csv('../datasets/imdb/train_movies_extra_data_new.csv', header=True, inferSchema=True)

In [4]:
"""
Loading the validation csv.
"""

validation = spark.read.option("escape", "\"").csv('../datasets/imdb/validation_extra_data.csv', header=True, inferSchema=True)

In [5]:
"""
Loading the test csv.
"""

test = spark.read.option("escape", "\"").csv('../datasets/imdb/test_extra_data.csv', header=True,
    inferSchema=True)

In [38]:
from pyspark.sql.functions import col, udf, when, split, regexp_replace, min, explode
from pyspark.sql.types import BooleanType, IntegerType
from pyspark.ml.feature import OneHotEncoder, StringIndexer

import re

def manage_awards(award):

    if award is not None:
        if re.search('win', award) or re.search('wins', award) or re.search('won', award):
            return 1
        else:
            return 0
    else:
        return False

my_udf = udf(manage_awards, IntegerType())

def preprocessing_method(df):

    """
    This method will convert to integer all the years and will put to categorical weather
    a movie won any awards or not.
    :param df: dataframe
    :return: return a preprocessed dataframe
    """

    """
    Here casting to integer all the numerical values
    """
    df = df.withColumn('startYear', col('startYear').cast('int'))
    df = df.withColumn('endYear', col('endYear').cast('int'))
    df = df.withColumn('numVotes', col('numVotes').cast('int'))

    """
    Preprocess boxoffice, and put the lowest value if it is null.
    """

    df = df.withColumn("boxoffice", regexp_replace(col("boxoffice"), "[$,]", "").cast("int"))
    min_boxoffice = df.agg(min('boxoffice')).collect()[0][0]
    df = df.withColumn("boxoffice", when(col("boxoffice").isNull(), min_boxoffice).otherwise(col("boxoffice")))

    """
    Preprocess the imdb votes.
    """

    df = df.withColumn('imdb_votes', regexp_replace(col('imdb_votes'), "[,]", "").cast('int'))

    """
    Awards transformed into True or False
    """

    df = df.withColumn('awards', my_udf(df['awards']))

    """
    Substitute numVotes value when null with the value of imdb votes
    """

    # scaling part

    """
    df = df.withColumn('numVotes', when(col('numVotes').isNull(), col('imdb_votes')).
                       otherwise(col('numVotes')))
    df = df.withColumn('startYear', when(col('startYear').isNull(), col('imdb_year')).
                       otherwise(col('startYear')))
    """

    """
    Transformations from string to lists.
    """

    df = df.withColumn('country', split(col('country'), ','))
    df = df.withColumn('genre', split(col('genre'), ','))
    df = df.withColumn('actors', split(col('actors'), ','))
    df = df.withColumn('language', split(col('language'), ','))

    df_exploded = df.select('*', explode('actors').alias('my_value'))

    unique_values = [str(row[0]) for row in df_exploded.select(explode('my_value')).distinct().collect()]
    df_pivoted = df_exploded.groupBy('id').pivot('my_value', unique_values).agg(when(col('my_value').isNotNull(), 1).otherwise(0))

    # join the pivoted dataframe with the original dataframe
    df = df.join(df_pivoted, 'id', 'left_outer')

    """
    Dropping the columns.
    """

    df = df.drop('_c0', 'primaryTitle', 'originalTitle', 'endYear', 'plot', 'language', 'rating', 'entry_type', 'production', 'imdb_votes', 'imdb_year')

    return df

In [39]:
"""
Preprocessing the training dataset.
"""

training_prep = preprocessing_method(training)

AnalysisException: cannot resolve 'explode(my_value)' due to data type mismatch: input to function explode should be array or map type, not string;
'Project [explode(my_value#3792) AS List()]
+- Project [_c0#17, tconst#18, primaryTitle#19, originalTitle#20, startYear#3519, endYear#3541, runtimeMinutes#23, numVotes#3563, label#25, genre#3725, plot#27, actors#3747, language#3769, country#3703, awards#3681, boxoffice#3636, rating#33, entry_type#34, production#35, imdb_votes#3658, imdb_year#37, my_value#3792]
   +- Generate explode(actors#3747), false, [my_value#3792]
      +- Project [_c0#17, tconst#18, primaryTitle#19, originalTitle#20, startYear#3519, endYear#3541, runtimeMinutes#23, numVotes#3563, label#25, genre#3725, plot#27, actors#3747, split(language#29, ,, -1) AS language#3769, country#3703, awards#3681, boxoffice#3636, rating#33, entry_type#34, production#35, imdb_votes#3658, imdb_year#37]
         +- Project [_c0#17, tconst#18, primaryTitle#19, originalTitle#20, startYear#3519,

In [17]:
"""
Preprocessing the validation dataset.
"""

validation_prep = preprocessing_method(validation)

In [18]:
"""
Preprocessing the test dataset.
"""

test_prep = preprocessing_method(test)

In [37]:
"""
Saving to csv the training dataset.
"""

training_prep.toPandas().to_csv('../datasets/models_dataset/train.csv', index=False)

In [27]:
"""
Saving to csv the validation dataset.
"""

validation_prep.toPandas().to_csv('../datasets/models_dataset/validation.csv', index=False)


In [28]:
"""
Saving to csv the test dataset.
"""

test_prep.toPandas().to_csv('../datasets/models_dataset/test.csv', index=False)